# Bandstructure of Monolayer Graphene with the Tight-Binding Model + Nearest-Neighbor Approximation

**Table of Contents**

1. [Initialization](#sec-1-initialization)
2. [Constants and Vectors Regarding Graphene Lattice Structure](#sec-2-constants)
3. [Tight-Binding Hamiltonian](#sec-3-hamiltonian)
   - [a) parameters](#sec-3a-parameters)
   - [b) Caclulation of the bloch Hamiltonian (the pz/pi orbitals and the complete as well)](#sec-3b-bloch)
4. [Bandstructure calculations and plots](#sec-4-bands)
   - [a) parameters](#sec-4a-parameters)
   - [b) 3d visualizer of k-space vs energy near K point](#sec-4b-3d)
   - [c) plotting along symmetric line traced along cyan path in BZ figure](#sec-4c-line)
   - [d) plot along zoomed zymmetric path](#sec-4d-zoom)


<a id="sec-1-initialization"></a>
___
## 1. Initialization


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import json

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 (needed for 3D)
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from scipy.ndimage import gaussian_filter1d


<a id="sec-2-constants"></a>
___
# 2. Constants and Vectors Regarding Graphene Lattice Structure
Nearest neighbor vecotrs, lattice vectors, etc...

In [14]:

# Lattice constant (angstrom)
a = 2.46    # lattice spacing in angstroms
a_cc = 1.42    # C-C spacing in angstrmos
# Convenience
pi = np.pi
sqrt3 = np.sqrt(3.0)
I = 1j  # imaginary unit

def vec(x, y):
    """Return a 2D numpy vector."""
    return np.array([float(x), float(y)], dtype=float)


# --- Nearest-neighbor vectors (angstrom) ---
d1 = a_cc * vec(1, 0)
d2 = a_cc * vec(-1/2, sqrt3/2)
d3 = a_cc * vec(-1/2, -sqrt3/2)

# --- Lattice vectors (angstrom) ---
a1 = a * vec( sqrt3/2, -0.5 )
a2 = a * vec( sqrt3/2,  0.5 )

# --- Reciprocal base vectors (1/angstrom) ---
# b_i · a_j = 2π δ_ij for this convention
b1 = (2*np.pi/a) * vec(1/np.sqrt(3), -1.0)
b2 = (2*np.pi/a) * vec(1/np.sqrt(3),  1.0)

# --- High-symmetry K points (1/angstrom) ---
K0  = (2*b1 +   b2) / 3
K0p  = (  b1 + 2*b2) / 3
K1  = (- b1 +   b2) / 3
K1p = -K0
K2= -K0p
K2p = -K1

# --- M points (1/angstrom) ---
M0 =  0.5 * b1
M1 =  0.5 * b2
M2 =  0.5 * (b1 + b2)
M3 = -M0
M4 = -M1
M5 = -M2

# K and M points as arrays

K_points = {
    "K0": K0, "K0'": K0p, "K1": K1, "K1'": K1p, "K2": K2, "K2'": K2p,
}
M_points = {
    "M0": M0, "M1": M1, "M2": M2,
    "M3": M3, "M4": M4, "M5": M5,
}
# Γ (g) point for convenience
g = vec(0.0, 0.0)




# 2.1 Brillouin Zone Visualization


In [15]:
hexagon = np.array([K0, K0p, K1, K1p, K2, K2p, K0])  # closed loop
fig = go.Figure()

# Brillouin zone hexagon
fig.add_trace(
    go.Scatter(
        x=hexagon[:, 0],
        y=hexagon[:, 1],
        mode="lines",
        line=dict(width=2),
        name="1st Brillouin Zone",
        hoverinfo="skip",
    )
)

# K points
fig.add_trace(
    go.Scatter(
        x=[p[0] for p in K_points.values()],
        y=[p[1] for p in K_points.values()],
        mode="markers+text",
        name="K points",
        marker=dict(size=10, symbol="diamond"),
        text=list(K_points.keys()),
        textposition="top center",
    )
)

# M0 point
fig.add_trace(
    go.Scatter(
        x=[M0[0]],
        y=[M0[1]],
        mode="markers+text",
        name="M points",
        marker=dict(size=9, symbol="square"),
        text=["M0"],
        textposition="bottom center",
    )
)

# g (center)
fig.add_trace(
    go.Scatter(
        x=[g[0]],
        y=[g[1]],
        mode="markers+text",
        name="Γ",
        marker=dict(size=11, symbol="circle"),
        text=["Γ"],
        textposition="top left",
    )
)

# Blue line: origin → K1 → M1 → origin
fig.add_trace(
    go.Scatter(
        x=[g[0], K0[0], M0[0], g[0]],
        y=[g[1], K0[1], M0[1], g[1]],
        mode="lines",
        name="Path",
        line=dict(color="cyan", width=2),
        hoverinfo="skip",
    )
)

# ================================
#  Layout / styling
# ================================
fig.update_layout(
    width=500,
    height=500,
    font=dict(family="DejaVu Sans", size=14, color="white"),
    template="plotly_dark",
    title="Graphene Brillouin Zone",
    xaxis_title="kₓ (Å⁻¹)",
    yaxis_title="kᵧ (Å⁻¹)",
    xaxis=dict(range=[-2, 2], autorange=False),
    yaxis=dict(range=[-2, 2], autorange=False),
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor="rgba(0,0,0,0.3)",
        borderwidth=0,
    ),
    margin=dict(l=60, r=40, t=60, b=60),
)

# Light grid so you can see symmetry nicely
fig.update_xaxes(showgrid=True, gridwidth=1, zeroline=True, range=[-2.5, 2], autorange=False)
fig.update_yaxes(showgrid=True, gridwidth=1, zeroline=True, range=[-2, 2], autorange=False)

# Equal aspect ratio to keep the figure square
fig.update_yaxes(scaleanchor="x", scaleratio=1)

fig.show()

___
# 3. Tight-Binding Hamiltonian

<a id="sec-3-hamiltonian"></a>

# Text: Orbital Hamiltonians (edit me)
Hamitonian btw any 2 orbitals: $ H_{nm} (\bold R) = \langle \phi_m (\bold 0) | \hat H | \phi_n (\bold R) \rangle$

Each carbon atom holds 4 atomic orbitals avaialbe to bond (s, px, py, pz). In graphene, each unit cell contains 2 carbon atoms so inter-atomic orbitals must consider 8 total initial atomic orbitals (4 from each atom): $\phi^A_s, \phi^A_{px}, \phi^A_{py}, \phi^A_{pz}, \phi^B_s, \phi^B_{px}, \phi^B_{py}, \phi^B_{pz}$

Hamiltonian, thus, must be specified: $ H^{\alpha \beta}_{nm} (\bold R) = \langle \phi^{\alpha}_m (\bold 0) | \hat H | \phi^{\beta}_n (\bold R) \rangle$

The complete orbital Hamiltonian matrix is, thus, $H(\bold R) = \begin{bmatrix} H^{AA} (\bold R)& H^{AB}(\bold R) \\ H^{BA}(\bold R) & H^{BB}(\bold R) \end{bmatrix}$

where each matrix $H^{\alpha \beta} (\bold R) $ takes the form $\large H^{\alpha \beta} (\bold R) = \begin{bmatrix} H^{AA}_{ss \sigma} (\bold R) & H^{AA}_{sp_x \sigma} (\bold R) & H^{AA}_{sp_y \sigma} (\bold R) & H^{AA}_{sp_z \sigma} (\bold R) \\ H^{AA}_{s p_x \sigma} (\bold R) & H^{AA}_{p_x p_x \pi} (\bold R) & H^{AA}_{p_x p_y \sigma} (\bold R) & H^{AA}_{p_x p_z \sigma} (\bold R) \\ H^{AA}_{s p_y \sigma} (\bold R) & H^{AA}_{p_x p_y \sigma} (\bold R) & H^{AA}_{p_y p_y \pi} (\bold R) & H^{AA}_{p_y p_z \sigma}(\bold R) \\ H^{AA}_{s p_z \sigma} (\bold R) & H^{AA}_{p_x p_z \sigma} (\bold R) & H^{AA}_{p_y p_z \sigma} (\bold R) & H^{AA}_{p_z p_z \pi} (\bold R) \end{bmatrix} $

## a) parameters

<a id="sec-3a-parameters"></a>

In [16]:
# Additional Parameters for Hamiltonian calculation 

# --- Slater–Koster parameters (eV) ---
Es     =  8.37
Ep     =  0.00
Vsssig = -5.71
Vspsig = 5.42
Vppsig = 6.20
Vpppi = -3.07

# --- Citation for S-K parameters ---
CITATION = "Rezaei, H., Phirouznia, A. Modified spin–orbit couplings in uniaxially strained graphene. Eur. Phys. J. B 91, 295 (2018). https://doi.org/10.1140/epjb/e2018-80663-2"

# --- Structure factors built from nearest neighbors ---
def _kdot(rkx, rky, r):
    """k·r with rk=(kx,ky), r=(x,y).
    rkx is the x values of the vectors rkx,
    rky is the y values of the vecotrs rkx,
    r is a 2d vector so i first convert it into an array
    """
    rkx = np.asarray(rkx, dtype=float)
    rky = np.asarray(rky, dtype=float)
    r = np.asarray(r, dtype=float)
    return np.asarray(rkx * r[0] + rky * r[1], dtype=float)

def f(kx, ky):
    """f(k) = e^{i k·d1} + e^{i k·d2} + e^{i k·d3}"""
    return (np.exp(1j*_kdot(kx, ky, d1)) + np.exp(1j*_kdot(kx, ky, d2)) + np.exp(1j*_kdot(kx, ky, d3)))

def g(kx,  ky):
    """g(k) = e^{i k·d2} - e^{i k·d3}"""
    return (np.exp(1j*_kdot(kx, ky, d2)) - np.exp(1j*_kdot(kx, ky, d3)))

def h(kx, ky):
    """h(k) = 2 e^{i k·d1} - e^{i k·d2} - e^{i k·d3}"""
    return (2.0*np.exp(1j*_kdot(kx, ky, d1)) - np.exp(1j*_kdot(kx, ky, d2)) - np.exp(1j*_kdot(kx, ky, d3)))



<a id="sec-3b-bloch"></a>

## b) Caclulation of the bloch Hamiltonian (the pz/pi orbitals and the complete as well)

In [17]:
# ---------- REFERENCES ---------------------

'''https://youtu.be/3b34QM15JTM?si=nFdSKuHe1AVsKyoq'''

# ---------- JUST THE PZ ORBITALS ------------

def H_bloch_pzpzpi(kx, ky) -> np.ndarray: #k is 2d vector
    """
    Return the 2x2 Bloch Hamiltonian at (kx, ky).
    Replace the contents of this function with your actual model.

    kx and ky are singualr k points

    1. create H_AB/H_BA
    2. concatenate H_AA + H_AB and H_BA + H_BB
    """
    fk = f(kx, ky)
    print(fk.shape)
    print(kx.shape)
    
    H = np.zeros(kx.shape + (2, 2), dtype=np.complex128)
    H[..., 0, 1] = fk
    H[..., 1, 0] = np.conj(fk)
    
    return np.linalg.eigvalsh(H)


# ---------- INCLUDING OTHER ORBITALS ------------
def H_AB_block(kx, ky) -> np.ndarray: # is k is 2d vector (kx, ky)
    fk = f(kx, ky)
    gk = g(kx, ky)
    hk = h(kx, ky)

    H = np.zeros(fk.shape + (4, 4), dtype=np.result_type(fk, 1j))

    # Fill entries explicitly; RHS broadcasts over the leading k-shape
    H[..., 0, 0] = fk * Vsssig
    H[..., 0, 1] = -sqrt3 * Vspsig / 2.0
    H[..., 0, 2] =  hk * Vspsig / 2.0
    # H[..., 0, 3] = 0

    H[..., 1, 0] =  sqrt3 * Vspsig / 2.0
    H[..., 1, 1] =  fk * (Vppsig + Vpppi) / 2.0 + hk * (Vppsig - Vpppi) / 4.0
    H[..., 1, 2] = -sqrt3 * gk * (Vppsig - Vpppi) / 4.0
    # H[..., 1, 3] = 0

    H[..., 2, 0] = -hk * Vspsig / 2.0
    H[..., 2, 1] = -sqrt3 * gk * (Vppsig - Vpppi) / 4.0
    H[..., 2, 2] =  fk * (Vppsig + Vpppi) / 2.0 - hk * (Vppsig - Vpppi) / 4.0
    # H[..., 2, 3] = 0

    # last row
    # H[..., 3, 0:3] = 0
    H[..., 3, 3] = fk * Vpppi

    return H
    # return np.array([
    #     [ fk * Vsssig,                 -sqrt3 * Vspsig / 2.0,          hk * Vspsig / 2.0,                         0.0 ],
    #     [  sqrt3 * Vspsig / 2.0,   fk * (Vppsig + Vpppi) / 2.0 + hk * (Vppsig - Vpppi) / 4.0,  -sqrt3 * gk * (Vppsig - Vpppi) / 4.0,  0.0 ],
    #     [ -hk * Vspsig / 2.0,      -sqrt3 * gk * (Vppsig - Vpppi) / 4.0,  fk * (Vppsig + Vpppi) / 2.0 - hk * (Vppsig - Vpppi) / 4.0,  0.0 ],
    #     [  0.0,                      0.0,                                0.0,                                      fk * Vpppi ]
    # ], dtype=complex)


def H_bloch(kx, ky) -> np.ndarray: #k is 2d vector
    """
    Return the 8x8 Bloch Hamiltonian at (kx, ky).
    Replace the contents of this function with your actual model.

    1. create H_AB/H_BA
    2. concatenate H_AA + H_AB and H_BA + H_BB
    """
    H_AA = np.diag([Es, Ep, Ep, Ep]).astype(complex)
    H_BB = H_AA.copy()
    
    H_AB = H_AB_block(kx, ky)
    H_BA = np.conj(np.swapaxes(H_AB, -1, -2))

    H = np.zeros(kx.shape + (8, 8), dtype=np.result_type(H_AB, 1j))
    H[..., 0:4, 0:4] = H_AA
    H[..., 4:8, 4:8] = H_BB
    H[..., 0:4, 4:8] = H_AB
    H[..., 4:8, 0:4] = H_BA

    return np.linalg.eigvalsh(H)


# ---- calculate eigenvalues for the pzpzpi hamiltonian  spanning the entire Brillouin zone----

def eigenenergies(kx: np.array, ky: np.array, H_func) -> np.ndarray: # 8 x N array
    """
    kx and ky are 1d or 2d arrays of kpoints
    1. if kx.shape = (n, m), return arrow of (n, m, 8)
    2. if kx.shape = (n,), return arrow of (n, 8)
    """
   
    
    N = len(kx)
    bands = np.array(H_func(kx, ky))
    return bands


# # ---- Example 1 : calculate just kpoint K0, M_0, \g ----
# k_values = np.array([K0, M0, vec(0.0,0.0)])
# kx, ky = k_values[:, 0], k_values[:, 1]
# E_bands = eigenenergies(kx, ky, H_bloch) # should return a 8 x 1 matrix
# print(E_bands)



<a id="sec-4-bands"></a>

___
## 4. Plot the eigenenergies (from just pi orbitals) in 2d space

<a id="sec-4a-parameters"></a>

## a) parameters

In [18]:
kx = np.linspace(-1.8, 1.8, 100)
ky = np.linspace(-1.8, 1.8, 100)
Kx, Ky = np.meshgrid(kx, ky)
eps = eigenenergies(Kx, Ky, H_bloch_pzpzpi)
fermi_level = 0.0
D1 = 0.0

data = {
    "kx": kx.tolist(),
    "ky": ky.tolist(),
    "eps": eps.tolist(),         # full band structure grid
    "fermi_level": float(fermi_level),
    "nbands": int(eps.shape[-1]),
    "shape": [eps.shape[0], eps.shape[1], eps.shape[2]]  # [Ny, Nx, nbands]
}


(100, 100)
(100, 100)


<a id="sec-4b-3d"></a>

In [19]:
# --- Write JSON file ---
with open("bands/1L_BZ_bands.json", "w") as f_json:
    json.dump(data, f_json, indent=2)

print("Saved 1L_BZ_bands.json")

Saved 1L_BZ_bands.json


## b) 3d visualizer of k-space vs energy near K point

In [20]:

# Compute eigenenergies if not already computed
if 'eps' not in locals():
    eps = eigenenergies(Kx, Ky, H_bloch)

# Extract bands (assuming 2 bands for monolayer)
Z0 = eps[:, :, 0]  # First band
Z1 = eps[:, :, 1]  # Second band

# ========= energy bounds ============
zmin = float(np.nanmin([Z0, Z1]))
zmax = float(np.nanmax([Z0, Z1]))
z_pad = 0.1 * (zmax - zmin)  # 10% padding
zmin = zmin - z_pad
zmax = zmax + z_pad

# Apply energy bounds (mask values outside range)
Z0_plot = Z0.copy()
Z1_plot = Z1.copy()
Z0_plot[(Z0_plot < zmin) | (Z0_plot > zmax)] = np.nan
Z1_plot[(Z1_plot < zmin) | (Z1_plot > zmax)] = np.nan

# --- Create Plotly figure ---
fig = go.Figure()

# --- Add band surfaces ---
fig.add_trace(go.Surface(
    x=Kx, y=Ky, z=Z0_plot,
    colorscale="viridis",
    opacity=0.85,
    showscale=False,
    # colorbar=dict(title="Energy (eV)", x=1.02),
    name="Band 1"
))
fig.add_trace(go.Surface(
    x=Kx, y=Ky, z=Z1_plot,
    colorscale="viridis_r",
    opacity=0.85,
    showscale=False,
    name="Band 2"
))

# --- Add Fermi level plane (if defined) ---
if 'fermi_level' in locals():
    Zplane = np.full_like(Kx, fermi_level)
    fig.add_trace(go.Surface(
        x=Kx, y=Ky, z=Zplane,
        colorscale="Reds_r",
        opacity=0.3,
        showscale=False,
        name=f"E_F = {fermi_level} eV"
    ))

# ========== style / annotations ==========
spacing = 0.5
start = spacing * np.floor(zmin / spacing)
stop = spacing * np.ceil(zmax / spacing)
zticks = np.arange(start / spacing, stop / spacing + 0.5) * spacing

# Generate x and y ticks
xyticks = np.linspace(-1.8, 1.8, 7)  # -3, -2, -1, 0, 1, 2, 3

fig.update_layout(
    font=dict(family="DejaVu Sans", size=14, color="white"),
    margin=dict(l=0, r=0, t=0, b=0),
    scene=dict(
        xaxis=dict(
            title=dict(text="kₓ (Å⁻¹)"),
            tickvals=xyticks,
            tickfont=dict(family="DejaVu Sans", size=12)
        ),
        yaxis=dict(
            title=dict(text="kᵧ (Å⁻¹)"),
            tickvals=xyticks,
            tickfont=dict(family="DejaVu Sans", size=12)
        ),
        zaxis=dict(
            title=dict(text="Energy (eV)"),
            range=[zmin, zmax],
            tickvals=zticks,
            tickfont=dict(family="DejaVu Sans", size=12)
        ),
        aspectmode="manual",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.2)
        )
    ),
    template="plotly_dark",
    width=800,
    height=800,
    title="Monolayer Graphene Energy Bands"
)

fig.show(config={"displayModeBar": False})


# c) plotting along symmetric line traced along cyan path in BZ figure

<a id="sec-4c-line"></a>

In [21]:
# --- Load kpath from kpath.json ---
with open("kpaths/kpath.json", "r") as file:
    kpath_lines = json.load(file)

# Get absolute k coordinates
kx_line = np.array(kpath_lines["kx"])
ky_line = np.array(kpath_lines["ky"])

# Get K point for converting to relative coordinates (continuum model: K is at origin)
K = np.array([kpath_lines["K"]["kx"], kpath_lines["K"]["ky"]])

# Convert to relative coordinates (q = k - K)
# Get segment information for breaks
segments = kpath_lines.get("segments", [])
if segments:
    breaks = [segments[0]["start"], segments[0]["end"], segments[1]["end"] - 1 if len(segments) > 1 else len(kx_line) - 1]
    breaks = [min(b, len(kx_line) - 1) for b in breaks]  # Ensure valid indices
    k_labels = ["Γ", "K", "M"]
else:
    breaks = [0, len(kx_line)//2, len(kx_line) - 1]
    k_labels = ["start", "K", "end"]

# --- Compute eigenenergies ---
# In continuum model, use absolute k coordinates for eigenenergies
energies = eigenenergies(kx_line, ky_line, H_func=H_bloch_pzpzpi)  # shape (N, nbands)
nbands = energies.shape[1]

# --- Compute cumulative path distance s (x-axis) ---
dx = np.diff(kx_line, prepend=kx_line[0])
dy = np.diff(ky_line, prepend=ky_line[0])
s = np.cumsum(np.hypot(dx, dy))

# --- Get break positions at symmetry points ---
break_positions = [s[b] for b in breaks if b < len(s)]
break_positions.append(s[-1])  # Always include the last point

# --- Plotly figure ---
fig = go.Figure()

fig.add_trace(go.Scatter(
        x=s, y=energies[:, 0],
        mode="lines",
        line=dict(width=1.2),
        name=f"Valence Band",
    ))

fig.add_trace(go.Scatter(
        x=s, y=energies[:, 1],
        mode="lines",
        line=dict(width=1.2),
        name=f"Conduction band",
    ))
# --- Vertical separators at symmetry points ---
for x in break_positions[1:-1]:  # Skip first and last
    fig.add_vline(x=x, line_dash="dash", line_color="gray", opacity=0.5)

# --- Fermi level (horizontal) ---
fig.add_hline(y=fermi_level, line_dash="dot", line_color="gray", opacity=0.4)

# --- Update x-axis with symmetry point labels ---
# Ensure we have labels for all break positions
ticktext = []
for i, pos in enumerate(break_positions):
    if i < len(k_labels):
        ticktext.append(k_labels[i])
    else:
        ticktext.append(f"{i}")

fig.update_xaxes(
    title_text=r"$k$ path",
    tickmode="array",
    tickvals=break_positions,
    ticktext=ticktext,
    showgrid=True
)

# --- Layout ---
fig.update_layout(
    title="Bandstructure along Γ → K → M path",
    xaxis_title=r"$k$ path",
    yaxis_title="Energy (eV)",
    template="plotly_dark",
    font=dict(family="DejaVu Sans"),
    width=600,
    height=500,
    margin=dict(l=60, r=20, t=60, b=60),
    legend=dict(x=0.02, y=0.98, bgcolor="rgba(0,0,0,0.3)"),
)

fig.show()


(600,)
(600,)


In [ ]:
line_data = {
    "kx_line": kx_line.tolist(),
    "ky_line": ky_line.tolist(),
    "energy": energies.tolist(),         # full band structure grid
    "fermi_level": fermi_level,
    "interlayer_potential": D1,
}

# --- Write JSON file ---
with open("bands/1L_line_bands.json", "w") as f_json:
    json.dump(line_data, f_json, indent=2)

print("Saved 1L_line_bands.json")

Saved 1L_line_bands.json


## d) plot along zoomed zymmetric path

<a id="sec-4d-zoom"></a>

In [23]:
# --- Load kpath from kpath.json ---
with open("kpaths/kpath_zoomed.json", "r") as file:
    kpath_lines_zoomed = json.load(file)

# Get absolute k coordinates
kx_line_zoomed = np.array(kpath_lines_zoomed["kx"])
ky_line_zoomed = np.array(kpath_lines_zoomed["ky"])

# Get K point for converting to relative coordinates (continuum model: K is at origin)
K = np.array([kpath_lines_zoomed["K"]["kx"], kpath_lines_zoomed["K"]["ky"]])

# Convert to relative coordinates (q = k - K)
# Get segment information for breaks
segments = kpath_lines_zoomed.get("segments", [])
if segments:
    breaks = [segments[0]["start"], segments[0]["end"], segments[1]["end"] - 1 if len(segments) > 1 else len(kx_line) - 1]
    breaks = [min(b, len(kx_line) - 1) for b in breaks]  # Ensure valid indices
    k_labels = ["Γ", "K", "M"]
else:
    breaks = [0, len(kx_line)//2, len(kx_line) - 1]
    k_labels = ["start", "K", "end"]

# --- Compute eigenenergies ---
# In continuum model, use absolute k coordinates for eigenenergies
energies = eigenenergies(kx_line_zoomed, ky_line_zoomed, H_func=H_bloch_pzpzpi)  # shape (N, nbands)
nbands = energies.shape[1]

# --- Compute cumulative path distance s (x-axis) ---
dx = np.diff(kx_line_zoomed, prepend=kx_line_zoomed[0])
dy = np.diff(ky_line_zoomed, prepend=ky_line_zoomed[0])
s = np.cumsum(np.hypot(dx, dy))

# --- Get break positions at symmetry points ---
break_positions = [s[b] for b in breaks if b < len(s)]
break_positions.append(s[-1])  # Always include the last point

# --- Plotly figure ---
fig = go.Figure()

fig.add_trace(go.Scatter(
        x=s, y=energies[:, 0],
        mode="lines",
        line=dict(width=1.2),
        name=f"Valence Band",
    ))

fig.add_trace(go.Scatter(
        x=s, y=energies[:, 1],
        mode="lines",
        line=dict(width=1.2),
        name=f"Conduction band",
    ))
# --- Vertical separators at symmetry points ---
for x in break_positions[1:-1]:  # Skip first and last
    fig.add_vline(x=x, line_dash="dash", line_color="gray", opacity=0.5)

# --- Fermi level (horizontal) ---
fig.add_hline(y=fermi_level, line_dash="dot", line_color="gray", opacity=0.4)

# --- Update x-axis with symmetry point labels ---
# Ensure we have labels for all break positions
ticktext = []
for i, pos in enumerate(break_positions):
    if i < len(k_labels):
        ticktext.append(k_labels[i])
    else:
        ticktext.append(f"{i}")

fig.update_xaxes(
    title_text=r"$k$ path",
    tickmode="array",
    tickvals=break_positions,
    ticktext=ticktext,
    showgrid=True
)

# --- Layout ---
fig.update_layout(
    title="Bandstructure 2d near K point",
    xaxis_title=r"$k$ path",
    yaxis_title="Energy (eV)",
    template="plotly_dark",
    font=dict(family="DejaVu Sans"),
    width=600,
    height=500,
    margin=dict(l=60, r=20, t=60, b=60),
    legend=dict(x=0.02, y=0.98, bgcolor="rgba(0,0,0,0.3)"),
)

fig.show()


(162,)
(162,)
